In [2]:
# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence

import speech_recognition as sr
# obtain path to "english.wav" in the same folder as this script
from os import path

#set the GOOGLE_APPLICATION_CREDENTIALS env variable for this notebook to use the text-to-speech API
import os

#import pandas to export the quotes to a file
import pandas as pd

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../Google Speech-to-Text key/HotS Soundboards-1ca9e2e5d0be.json"

#set current hero
curr_hero = "Tyrael"

In [6]:
# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)


def split_apart_recording(audio):
    print("starting function")
    # Split track where the silence is 2 seconds or more and get chunks using 
    # the imported function.
    chunks = split_on_silence (
        # Use the loaded audio.
        audio, 
        # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
        min_silence_len = 1200,
        # Consider a chunk silent if it's quieter than -16 dBFS.
        # (You may want to adjust this parameter.)
        silence_thresh = -44
    )
    print("split on silence already")
    num_chunks = len(chunks)
    print(num_chunks)

    # Process each chunk with your parameters
    for i, chunk in enumerate(chunks):
        print("on chunk {}".format(i+1))
        # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
        silence_chunk = AudioSegment.silent(duration=500)

        # Add the padding chunk to beginning and end of the entire chunk.
        audio_chunk = silence_chunk + chunk + silence_chunk
#         audio_chunk=chunk

        # Normalize the entire chunk. 
        #NOTE: Add back later if necessary
        normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

        # Export the audio chunk with new bitrate.
        print("Exporting chunk{}.wav.".format(i+1))
        #could also use normalized chunk if necessary and that feature is used
        normalized_chunk.export(
            "../output/{}/chunk{}.wav".format(curr_hero, i+1),
            bitrate = "192k",
            format = "wav"
        )
        print("Exported chunk {}".format(i+1))
    
    return num_chunks

In [7]:
# Load your audio.
audio = AudioSegment.from_mp3("../source_audio/{}/Tyrael FULL Quotes - Heroes of the Storm.mp3".format(curr_hero))
print(audio)

In [8]:
num_clips = split_apart_recording(audio)

starting function
split on silence already
159
on chunk 1
Exporting chunk1.wav.
Exported chunk 1
on chunk 2
Exporting chunk2.wav.
Exported chunk 2
on chunk 3
Exporting chunk3.wav.
Exported chunk 3
on chunk 4
Exporting chunk4.wav.
Exported chunk 4
on chunk 5
Exporting chunk5.wav.
Exported chunk 5
on chunk 6
Exporting chunk6.wav.
Exported chunk 6
on chunk 7
Exporting chunk7.wav.
Exported chunk 7
on chunk 8
Exporting chunk8.wav.
Exported chunk 8
on chunk 9
Exporting chunk9.wav.
Exported chunk 9
on chunk 10
Exporting chunk10.wav.
Exported chunk 10
on chunk 11
Exporting chunk11.wav.
Exported chunk 11
on chunk 12
Exporting chunk12.wav.
Exported chunk 12
on chunk 13
Exporting chunk13.wav.
Exported chunk 13
on chunk 14
Exporting chunk14.wav.
Exported chunk 14
on chunk 15
Exporting chunk15.wav.
Exported chunk 15
on chunk 16
Exporting chunk16.wav.
Exported chunk 16
on chunk 17
Exporting chunk17.wav.
Exported chunk 17
on chunk 18
Exporting chunk18.wav.
Exported chunk 18
on chunk 19
Exporting chun

In [10]:
def audio_transcription(num_chunks):
    # use the audio file as the audio source
    r = sr.Recognizer()
    
    for i in range(1, num_chunks+1):
        with sr.AudioFile("../output/{}/chunk{}.wav".format(curr_hero, i)) as source:
            chunk = r.record(source)  # read the entire audio file

        # recognize speech using Sphinx
        try:
            print("Sphinx thinks you said -> " + r.recognize_sphinx(chunk))
        except sr.UnknownValueError:
            print("Sphinx could not understand audio")
        except sr.RequestError as e:
            print("Sphinx error; {0}".format(e))

In [11]:
audio_transcription(num_clips)

Sphinx thinks you said -> and it's mortal
Sphinx thinks you said -> what do you think
Sphinx thinks you said -> i'm sorry did you say something i was busy thinking about justice
Sphinx thinks you said -> you want more hope you never know what your curiosity likes of which
Sphinx thinks you said -> saint louis just to sort of do love this war and you'll get lost to be doomed to imagine what to visit the bullshit these days and ha-ha ha ha ha ha ha ha ha and where are we do and where does it didn't work
Sphinx thinks you said -> just because i'm mortal bosnia one to wait around forever
Sphinx thinks you said -> the person's body glaring at you right now
Sphinx thinks you said -> in heaven everything is fine
Sphinx thinks you said -> you can always sell and angels have big when their wings locker
Sphinx thinks you said -> perfect gemma activated
Sphinx thinks you said -> you're quite proficient at likud do ya bills or you might be right up your alley
Sphinx thinks you said -> ever taco be

Sphinx thinks you said -> let's destroy the enemy's call we should do it is most america
Sphinx thinks you said -> let's attack is forged
Sphinx thinks you said -> i suggest we do have to do is watch draw
Sphinx thinks you said -> we should capture this objective we should capture this objectives we should collect these
Sphinx thinks you said -> yeah but these quickly friends
Sphinx thinks you said -> be warned
Sphinx thinks you said -> the germans nearby
Sphinx thinks you said -> it would be wise to defend you
Sphinx thinks you said -> the third on call
Sphinx thinks you said -> even the cohorts
Sphinx thinks you said -> protect this ought to hades in our interests to do their duty structure give them this with your lives heroes we must defend this now
Sphinx thinks you said -> perhaps we should do that and it is watched are
Sphinx thinks you said -> shatter their structures foundations we should demolished the structure this one must be removed
Sphinx thinks you said -> we should mak

In [9]:
def audio_transcription_google(num_chunks):
    r = sr.Recognizer()
    output_quotes = []
    
    for i in range(1, num_chunks+1):
        with sr.AudioFile("../output/{}/chunk{}.wav".format(curr_hero, i)) as source:
                    chunk = r.record(source)

        #setup google speech-to-text transcription
        # recognize speech using Google Speech Recognition
        try:
            # for testing purposes, we're just using the default API key
            # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
            # instead of `r.recognize_google(audio)`
            print("Currently on quote #{}".format(i))
            curr_quote = r.recognize_google(chunk)
            print("Google Speech Recognition thinks you said -> " + curr_quote)
            output_quotes.append(curr_quote)
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            output_quotes.append("")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
            output_quotes.append("")
            
    return output_quotes

In [10]:
transcribed_quotes = audio_transcription_google(num_clips)

Currently on quote #1
Google Speech Recognition thinks you said -> greetings mortal
Currently on quote #2
Google Speech Recognition thinks you said -> what do you need
Currently on quote #3
Google Speech Recognition thinks you said -> I'm sorry did you say something I was busy thinking about Justice
Currently on quote #4
Google Speech Recognition thinks you said -> be warned mortal you never know what your curiosity might unleash
Currently on quote #5
Google Speech Recognition thinks you said -> you cannot even begin to imagine what you set in motion this day
Currently on quote #6
Google Speech Recognition thinks you said -> just because I'm an immortal doesn't mean I want to wait around forever
Currently on quote #7
Google Speech Recognition thinks you said -> if I got a face I'd be glaring at you right now
Currently on quote #8
Google Speech Recognition thinks you said -> in heaven everything is fine
Currently on quote #9
Google Speech Recognition thinks you said -> you can always te

Google Speech Recognition thinks you said -> no
Currently on quote #82
Google Speech Recognition thinks you said -> bael druins light
Currently on quote #83
Google Speech Recognition thinks you said -> cold fiend
Currently on quote #84
Google Speech Recognition thinks you said -> evil will be rooted out
Currently on quote #85
Google Speech Recognition thinks you said -> Justice be done
Currently on quote #86
Google Speech Recognition could not understand audio
Currently on quote #87
Google Speech Recognition thinks you said -> you cannot stand before el'druin
Currently on quote #88
Google Speech Recognition thinks you said -> I will bring order to chaos I cannot move
Currently on quote #89
Google Speech Recognition thinks you said -> well done friends keep up the fight we have done it
Currently on quote #90
Google Speech Recognition thinks you said -> we are closing in on them do not lose hope
Currently on quote #91
Google Speech Recognition could not understand audio
Currently on quot

In [11]:
curr_quotes = pd.Series(transcribed_quotes)
curr_quotes.to_csv("../output/{}/transcription_arr.csv".format(curr_hero))

In [12]:
#change the file names for the hero to the transcribed quotes

for i, quote in enumerate(transcribed_quotes):
    chunk_num = i+1
    os.rename("../output/{}/chunk{}.wav".format(curr_hero, chunk_num), 
              "../output/{}/{:0>3d}_{}.wav".format(curr_hero, chunk_num, quote))
    
print("{} output quotes renamed!".format(curr_hero))
    

Tyrael output quotes renamed!
